# Model Playground

In [1]:
# Imports
import json
import os
import torch
import numpy as np
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim

import pretty_midi
from model import Net
from midi_to_piano_roll import midi_to_piano_roll
from loss import blur_loss

import time
import pickle

In [2]:
# Load our checkpoint
ckpt_path = "/media/allentao/One Touch/APS360/ckpts/jul30_lr0.01_before_postprocess/checkpoint_epoch116.pt"
state = torch.load(ckpt_path)
model = Net(width = 3, batch_size = 1)
model.load_state_dict(state['state_dict'])

<All keys matched successfully>

In [3]:
# Load our data sample
# song midi
IN_FOLDER = '../data/clean_data/'
song_midi_path = IN_FOLDER + "1_0_song.midi"

# convert to piano roll
song_piano_roll = midi_to_piano_roll(song_midi_path)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
# Run inference
model = model.to(device)
out = model(song_piano_roll.unsqueeze(0).to(device))
out.shape

torch.Size([1, 22705, 128])

In [6]:
out

tensor([[[6.1290e-05, 3.2980e-05, 5.2028e-05,  ..., 2.9458e-05,
          3.6429e-05, 3.0528e-05],
         [5.2626e-05, 4.9022e-05, 4.6929e-05,  ..., 2.7359e-05,
          3.0954e-05, 3.3607e-05],
         [3.9965e-05, 2.1767e-05, 9.3727e-05,  ..., 1.1745e-04,
          3.2476e-05, 3.3984e-05],
         ...,
         [4.6474e-05, 7.1146e-05, 5.9267e-05,  ..., 5.9515e-05,
          8.9962e-05, 3.0244e-05],
         [4.7898e-05, 2.4365e-05, 3.2893e-05,  ..., 2.5035e-05,
          5.0519e-05, 6.4881e-05],
         [4.9933e-05, 2.1932e-05, 3.4271e-05,  ..., 3.0191e-05,
          6.0375e-05, 2.8528e-05]]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)

In [8]:
# Convert piano roll to midi file
from pianoroll_to_midi import piano_roll_to_midi

midi_out = piano_roll_to_midi(out.to("cpu"))
midi_out.save('test.mid')

LookupError: Unknown message type 'set_tempo'